# Hiperparámetros

Son los parámetros que condicionan el entrenamiento de un modelo, y son una pieza clave para encontrar la mejor configuración disponible. Esta labor requiere que seamos capaces de buscar en un amplio espectro de opciones y aprovechar al máximo las capacidades de nuestras máquinas.

En el ejemplo abajo veremos cómo la búsqueda se realiza para la maya que representan los parámetros y retorna el modelo que mejor resultado de empleando validación cruzada.

In [3]:
from sklearn.datasets import load_wine
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Datos
X, y = load_wine(return_X_y=True)

# Parámetros a ajustar
parameters = {
    'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
    'C': [0.001, 0.1, 0.5, 1, 5, 10, 100],
    'degree': [1,2,3,4,5,6,7],
    'gamma': ['scale', 'auto']
}

# Modelo base
svc = SVC()

# Búsqueda
clf = GridSearchCV(estimator = svc,
                  param_grid = parameters,
                  n_jobs = -1, # todos los hilos
                  cv = 10,
                  scoring="accuracy")

clf.fit(X, y)

,estimator,SVC()
,param_grid,"{'C': [0.001, 0.1, ...], 'degree': [1, 2, ...], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf', ...]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,C,0.1


In [4]:
clf.score(X, y)

0.9775280898876404

In [5]:
print("Parámetros ganadores ", clf.best_params_)
print("Valor promedio de la mejor evaluación cruzada ", clf.best_score_)

Parámetros ganadores  {'C': 0.1, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}
Valor promedio de la mejor evaluación cruzada  0.9666666666666668


In [6]:
clf.best_estimator_

,C,0.1
,kernel,'linear'
,degree,1
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


Podemos realizar tantos cambios como queramos en nuestras búsquedas, incluyendo el modelo a emplear.

In [16]:
# Obviamos algunos warnings por claridad

import warnings
warnings.filterwarnings("ignore")

In [17]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Semilla
np.random.seed(0)

# Pipeline base
pipe = Pipeline(steps=[("scaler", StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# Parámetros para buscar con reg logarítmica
logistic_params = {
    'classifier': [LogisticRegression(max_iter=1000, solver='saga'), LogisticRegression(max_iter=100, solver='saga')],
    'classifier__penalty': ['l1', 'l2']
}

# Búsqueda con árboles
random_forest_params = {
    'scaler': [StandardScaler(), MinMaxScaler()],
    'classifier': [RandomForestClassifier()],
    'classifier__max_depth': [2,3,4]
}

# Con SVC
svm_param = {
    'classifier': [SVC()],
    'classifier__C': [0.001, 0.1, 0.5, 1, 5, 10, 100],
}

# Unimos todo en una lista
search_space = [
    logistic_params,
    random_forest_params,
    svm_param
]

# Y creamos nuestra búsqueda
clf = GridSearchCV(estimator = pipe,
                  param_grid = search_space,
                  cv = 5,
                  n_jobs=-1,
                  verbose=False)

clf.fit(X, y)

,estimator,Pipeline(step...lassifier())])
,param_grid,"[{'classifier': [LogisticRegre...solver='saga'), LogisticRegre...solver='saga')], 'classifier__penalty': ['l1', 'l2']}, {'classifier': [RandomForestClassifier()], 'classifier__max_depth': [2, 3, ...], 'scaler': [StandardScaler(), MinMaxScaler()]}, ...]"
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,False
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


In [18]:
clf.score(X, y)

1.0

## Optuna

En los últimos años ha proliferado mucho el uso de [Optuna](https://optuna.org/) una librería específica para la búsqueda de hiperparámetros. Más allá del GridSearch y RandomSearch implementa búsquedas algo más eficientes en tiempo y con más opciones de ajuste.

Nos da la opción de almacenar nuestros experimentos para registrar los resultados a posteriori.

In [56]:
import optuna

study = optuna.create_study(
    storage="sqlite:///db.sqlite3",  # Almacenamiento
    study_name="vino-optimo",
    direction='maximize',
    sampler=optuna.samplers.TPESampler(), #CmaEsSampler(), # Probad con otros
    load_if_exists=True
)

[I 2025-07-27 10:18:56,756] A new study created in RDB with name: vino-optimo


Optuna permite visualizar de forma gráfica los resultados del estudio mediante el comando

```sh
optuna-dashboard sqlite:///db.sqlite3
```

pero para los que usamos VSCode, existe una [extensión](https://marketplace.visualstudio.com/items?itemName=Optuna.optuna-dashboard#overview) para que no tengáis que lanzar el comando anterior y os sea más cómodo.

Una de las bondades de Optuna es que nos permite explorar de forma eficiente el espacio de solución, con opciones más allá de Grid Y Random:

* Grid Search
* Random Search
* Tree-structured Parzen Estimator algorithm
* CMA-ES based algorithm
* Gaussian process-based algorithm

https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/003_efficient_optimization_algorithms.html

Crearemos nuestro estudio para analizar los parámetros que mejor resultan con modelo de boosting basado en árboles de decisión.

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import HistGradientBoostingClassifier

# Función objetivo
def objective(trial):

    # Parámetros base para optuna
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1.0, log=True),
        "l2_regularization" : trial.suggest_int("l2_regularization", 0.0, 1.0),
    }

    # Modelo
    clf = HistGradientBoostingClassifier(
        **param # unpacking
    )
    clf.fit(X_train, y_train)
    
    # Obtenemos la probabilidad asociada a la clase
    pred_proba = clf.predict_proba(X_test)

    # Área bajo la curva
    return roc_auc_score(y_true=y_test, y_score=pred_proba, multi_class='ovr') # Multiclase 

### Clasificación probabilística

Muchas veces los modelos nos devolverán su categorización y un porcentaje asociado a la confianza de la etiqueta asignada, o incluso, basándonos en modelos de regresión veremos que lo que arrojan es un valor numérico entre 0 y 1 y dependiendo el umbral que seleccionemos seremos capaces de priorizar una mayor o menor detección de positivos en favor de más o menos falsos negativos.

La curva ROC (Receiver Operating Characteristic) es una representación gráfica de cómo de bien funciona nuestro modelo ante distintos umbrales de detección para clasificación binaria. Se empleó en la segunda guerra mundial para analizar la capacidad de detección de distintos radares a la hora de detectar naves Japonesas (de ahí su nombre).

![curva](../../assets/images/curvaroc.png)

Si dibujamos el área bajo la curva podemos obtener una métrica de bondad, de manera que un 1 en el AUC (Area under the Curve) implica la perfección de nuestro modelo. Esta métrica funciona particularmente bien ante conjuntos de datos poco balanceados.

In [59]:
# Lanzamos el estudio
study.optimize(objective, n_trials=100)

[I 2025-07-27 10:19:01,322] Trial 0 finished with value: 0.9794372294372294 and parameters: {'learning_rate': 2.4615228430347585e-08, 'l2_regularization': 0}. Best is trial 0 with value: 0.9794372294372294.
[I 2025-07-27 10:19:01,513] Trial 1 finished with value: 0.9794372294372294 and parameters: {'learning_rate': 0.0006916891483698394, 'l2_regularization': 0}. Best is trial 0 with value: 0.9794372294372294.
[I 2025-07-27 10:19:01,680] Trial 2 finished with value: 0.9779491341991342 and parameters: {'learning_rate': 4.7759736115728626e-06, 'l2_regularization': 1}. Best is trial 0 with value: 0.9794372294372294.
[I 2025-07-27 10:19:01,834] Trial 3 finished with value: 0.9779491341991342 and parameters: {'learning_rate': 1.820681666573094e-07, 'l2_regularization': 1}. Best is trial 0 with value: 0.9794372294372294.
[I 2025-07-27 10:19:02,064] Trial 4 finished with value: 0.9794372294372294 and parameters: {'learning_rate': 5.4422182180117296e-08, 'l2_regularization': 0}. Best is trial 0

In [60]:
study.best_params

{'learning_rate': 0.5585893981679334, 'l2_regularization': 1}

In [61]:
study.best_trial

FrozenTrial(number=8, state=1, values=[1.0], datetime_start=datetime.datetime(2025, 7, 27, 10, 19, 2, 608408), datetime_complete=datetime.datetime(2025, 7, 27, 10, 19, 2, 719713), params={'learning_rate': 0.5585893981679334, 'l2_regularization': 1}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'l2_regularization': IntDistribution(high=1, log=False, low=0, step=1)}, trial_id=9, value=None)